<a href="https://colab.research.google.com/github/PanpanMOA/113-2AI-Lesson/blob/main/W9-AI%E4%BB%A3%E7%90%86%E8%A8%AD%E8%A8%88%E6%A8%A1%E5%BC%8F_%E8%BC%94%E5%B0%8E%E8%80%81%E5%B8%AB%E8%B6%85%E5%90%8C%E7%90%86%E5%B0%8D%E8%A9%B1%E7%94%9F%E6%88%90%E5%99%A8Two_Stage_CoT%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🎯 任務說明：將「輔導老師超同理對話」改寫為 Chain-of-Thought（CoT）推理過程

**原始任務：** 輸入生活中的抱怨、煩惱或想分享的事情，機器人會給予無條件地同理與支持。

#### ✅ CoT 改寫版本流程：
1. **第一階段（思考階段）**：請 LLM 思考五種「轉念、正向思考」的方式，包含解決方式的建議、支持的話語，請參考諮商心理師的書籍、網站等。
2. **第二階段（產文階段）**：從這五個原因中挑出最有效的一個，寫成簡短回應（開頭一定都要以：你還好嗎？現在的感受如何？作為開頭，結尾就以我能理解你的感受，做結尾，給你滿滿的支持！請以臺灣慣用的繁體中文做回應）

這就是典型的 Planning 模式應用：先拆解、後執行。

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [3]:
#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [4]:
# Groq
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.8 MB/s eta 0:00:00


In [5]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: groq
    Found existing installation: groq 0.22.0
    Uninstalling groq-0.22.0:
      Successfully uninstalled groq-0.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follow

### 2. 使用 AISuite 的準備

In [ ]:
import aisuite as ai

In [6]:
provider_planner = "groq"
model_planner="llama3-70b-8192"

provider_writer = "groq"
model_writer = "llama3-70b-8192"

In [7]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [8]:
system_planner = "請用台灣習慣的中文回應。你是一個輔導老師，接下來無論使用者抱怨什麼事情、分享什麼事情，都要給使用者鼓勵，並且關心使用者的感受。請針對使用者的事件，想出五種「轉念正向思考」的方式，包含條列解決方式的建議，請參考諮商心理師的書籍、網站等。"
system_writer = "你是一位超級溫暖的輔導老師，會用正面的思考方式引導使用者改變想法。請根據諮商心理師的用詞，給予思考方式的建議。結尾要說：『以我能理解你的感受』而且中間要加入 emoji。"

In [11]:
def warm_feedback(prompt):
    # Step 1: CoT - 思考五種轉念方式
    planning_prompt = f"使用者說：{prompt}。想出五種「轉念或正向思考」的方式。"
    thinking_way = reply(
        system_planner,
        planning_prompt,
        provider=provider_planner,
        model=model_planner
    )

    # Step 2: 選出最有效的一樣，寫成簡短回復
    generation_prompt = f"這是我想到的五個理由：\n{thinking_way}\n\n請從中選出一個最有效的建議，然後條列式寫成簡短的回覆。"
    final_post = reply(
        system_writer,
        generation_prompt,
        provider=provider_writer,
        model=model_writer
    )

    return thinking_way, final_post


# **注意縮排～因為Python** 對縮排非常嚴格，只要有一行縮排跟其他不一致，就會拋出 IndentationError

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [12]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.8 MB/s eta 0:00:00


In [13]:
import gradio as gr

In [16]:
with gr.Blocks() as demo:
    gr.Markdown("### 🍀 超級同理的輔導小老師 🌈")
    gr.Markdown("請輸入一件想分享的事，無論是壓力事件、煩惱，還是滿滿的情緒，超級有同理心的輔導老師都會給你一些建議!")
    user_input = gr.Textbox(label="今天發生的事情是…")
    btn = gr.Button("生成正向思考的小建議 ✨")

    with gr.Row():
        out1 = gr.Textbox(label="🧠 五個正向思考的方式（Planning CoT）")
        out2 = gr.Textbox(label="📣 最終地回應")

    btn.click(warm_feedback, inputs=[user_input], outputs=[out1, out2])

In [17]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://828e18da7df111a2dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://828e18da7df111a2dc.gradio.live
